In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx

from numba import jit, prange, jitclass
from numba.types import int32
from time import time
from pprint import pprint
from copy import deepcopy as copy


%matplotlib inline

## Read data files

**Note** We are keeping only the 50 states. DC not included. Self loops not included

In [4]:
def read_US_states(fname):
    states_abb_dict = {}
    states_abb_rev_dict = {}
    states_abb_ord_list = []
    with open(fname, "r") as f:
        for line in f:
            line = line.strip().split(",")
            name = line[0]
            abbr = line[1]
            states_abb_dict[name] = (abbr, len(states_abb_ord_list))
            states_abb_rev_dict[abbr] = name
            states_abb_ord_list.append(abbr)
    
    return states_abb_dict, states_abb_rev_dict, states_abb_ord_list

def read_travel_network(fname, states_abb_dict, states_abb_rev_dict, normalization=1000):
    num_states = 50

    adjacency_list = {}
    A = np.zeros((num_states, num_states))

    for abb in states_abb_rev_dict:
        adjacency_list[abb] = []

    with open(fname, "r") as f:
        for idx, line in enumerate(f):
            if idx == 0:
                continue
            line = line.strip().split(",")
            orig = line[0]
            dest = line[1]
            weight = float(line[2])*10 / normalization
            if orig == dest:
                continue
            try:
                orig_abb, orig_idx = states_abb_dict[orig]
                dest_abb, dest_idx = states_abb_dict[dest]
                adjacency_list[orig_abb].append((dest_abb, weight))
                A[orig_idx][dest_idx] = weight
            except KeyError:
                pass
    
    return adjacency_list, A

def read_deaths_data(fname):
    df = pd.read_csv(fname)
    sum_df_d = df.groupby(['State', 'Year', 'Quarter'])[['DeathsFromPneumoniaAndInfluenza']].sum().T.to_dict()
    deaths_dict = {}
    for key, deaths in sum_df_d.items():
        state, year, quarter = key
        if state not in deaths_dict:
            deaths_dict[state] = {}
        if year not in deaths_dict[state]:
            deaths_dict[state][year] = {}
        deaths_dict[state][year][quarter] = deaths["DeathsFromPneumoniaAndInfluenza"]
    return deaths_dict

def read_population_dict(fname):
    population_dict = {}
    with open(fname, "r") as f:
        for idx, line in enumerate(f):
            if idx == 0:
                continue
            line = line.strip().split(",")
            state = line[1]
            population_dict[state] = {}
            year = 2009
            for idx, pop in enumerate(line[2:]):
                population_dict[state][year + idx] = int(pop)
    return population_dict

## Simulation


1. Randomly infect people in each state with probability `p_inf`
    - Calculate fraction of infected people
2. Air travel the people
    - Fraction of infected air travelers is the same as origin state
    - Each person has a chance to infect every other person in the new state with probability `p_transfer`
3. In each state, either recover, kill, or remain infected the people there with probability `p_rec, p_die, p_stay`
4. Increment, or decrement, population between years (not quarters)

Totally, there are 4 parameters: `p_inf`, `p_transfer`, `p_rec`, `p_die`

Note `p_stay = 1 - p_rec - p_die`

Can make into a 3 parameter model by setting `p_stay = 0` i.e., between quarters, people either heal completely, or die.

### Goal: Find the best parameters that represent the number of deaths observed

In [97]:
spec = [
    ('num_total', int32),
    ('num_infected', int32),
    ('num_deceased', int32)
]

@jitclass(spec)
class State:
    def __init__(self, pop):
        self.num_total = pop
        self.num_infected = 0
        self.num_deceased = 0
    
    def infected_fraction(self):
        return self.num_infected / self.num_total

### Kernels for simulation

In [120]:
# @jit(nopython=True, parallel=True)
def travel_and_infect_kernel(A, states, p_transfer, verbose=False):
    N = A.shape[0]
#     new_states = copy(states)
    new_states = []
    for i in range(N):
        new_state = State(states[i].num_total)
        new_state.num_infected = states[i].num_infected
        new_state.num_deceased = states[i].num_deceased
        new_states.append(new_state)
    
    for i in range(N):
        
        # Let X be number of successful infections.
        # X is Binomial(n, p_transfer) where n is number of travelers.
        # We want X >= 1 for each person at destination i.e.,
        # at least one successful infection for each person at destination.
        # So, calculate p' = P(X >= 1) = 1 - P(X = 0) = 1 - (1-p_transfer)^n.
        # p' is the probability a person at destination gets infected.
        # This reduces problem to calculate a new r.v. Y.
        # Y is Binomial(m, p'), where m is the uninfected population of destination
        
        infected_fraction = states[i].infected_fraction()
        travel_pop_inf = A[i, :] * infected_fraction
        ccdf = 1 - (1 - p_transfer)**travel_pop_inf
        
        for j in range(N):
            if i == j:
                continue
            dest = states[j]
            new_states[j].num_infected += np.random.binomial(dest.num_total - dest.num_infected, ccdf[j])
        
    return new_states

@jit
def recover_kernel(states, SIR_params):
    for i in range(len(states)):
        x = np.random.uniform(0, 1, states[i].num_infected)
        recovered = len(np.asarray(x < SIR_params[0]).nonzero()[0])
        dead = len(np.asarray(x <= SIR_params[2]).nonzero()[0])
        states[i].num_infected -= (dead + recovered)
        states[i].num_deceased += dead
#         states[i] = state
    return states

def remove_deceased_kernel(states):
    num_deceased = []
    for i, state in enumerate(states):
        num_deceased.append(state.num_deceased)
        state.num_total = max(state.num_total - state.num_deceased, 0)
        state.num_deceased = 0
        states[i] = state
    return states, num_deceased
    
def inject_population_kernel(states, new_population):
    # new_population is somehow ordered
    for i, state in enumerate(states):
        current_pop = state.num_total
        nextgen_pop = new_population[state_id]
        extra_peeps = nextgen_pop - current_pop
        if extra_peeps > 0:
            state.num_total += extra_peeps
        elif extra_peeps < 0:
            # population actually decreased... remove infected and susceptible at same rate
            extra_peeps = -extra_peeps
            if extra_peeps // 2 >= state.num_infected:
                state.num_total = state.num_total - (extra_peeps - state.num_infected)
                state.num_infected = 0
            else:
                state.num_infected -= (extra_peeps - extra_peeps//2)
                state.num_total -= extra_peeps
        states[i] = state
    return states

def random_infection_kernel(states, p_inf):
    for i, state in enumerate(states):
        if state.num_total < state.num_infected:
            continue
        infected = np.random.binomial(state.num_total - state.num_infected, p_inf)
        state.num_infected += infected
        states[i] = state
    return states

### Simple 5 x 5 model

In [121]:
# Fix parameters of the model
p_inf = 0.1
p_transfer = 0.1
p_rec = 0.99
p_die = 7.540044190323758e-05
p_stay = 1 - p_rec - p_die
SIR = [p_rec, p_stay, p_die]

# Fix population and travel networks
N = 5
num_years = 3

A_dict = {}
for year in range(num_years):    
    A_dict[year] = {}
    for quarter in range(4):
        A = np.random.randint(1, 20, (N, N))
        for i in range(N):
            A[i, i] = 0
        A_dict[year][quarter] = A

all_population = {}
epsilon_plus = 50
epsilon_minus = -10
for idx, year in enumerate(range(num_years)):
    all_population[year] = []
    if idx == 0:
        for state_id in range(N):
            all_population[year].append(np.random.randint(10, 25) * 10)
    else:
        for state_id in range(N):
            all_population[year].append(all_population[year-1][state_id] +
                                        np.random.randint(epsilon_minus, epsilon_plus))

In [122]:
# Instantiate a random infection
states = []
year_0 = 0
for state_id in range(N):
    state = State(all_population[year_0][state_id])
    state.num_infected += np.random.binomial(state.num_total, p_inf)
    states.append(state)

# Propagate infection
num_deceased = {}

# For each year:
for year in range(num_years):
    num_deceased[year] = {}
    # For each quarter:
    for quarter in range(4):
        # 1. travel and infect
        # 2. Recover
        # 3. Remove deceased and store it
        # 4. Randomly infect
        states = travel_and_infect_kernel(A_dict[year][quarter], states, p_transfer)
        states = recover_kernel(states, SIR)
        
        states, dead_peeps = remove_deceased_kernel(states)
        num_deceased[year][quarter] = dead_peeps
        
        states = random_infection_kernel(states, p_inf)
    
    # Inject population
    try:
        pop_vec = inject_population_kernel(states, all_population[year+1])
    except KeyError:
        # We are at the end of our data
        pass

c:\python37\lib\site-packages\numba\ir_utils.py:1969: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected list' found for argument 'SIR_params' of function 'recover_kernel'.

For more information visit http://numba.pydata.org/numba-doc/latest/reference/deprecation.html#deprecation-of-reflection-for-list-and-set-types

File "<ipython-input-120-39d03efb684e>", line 36:
@jit
def recover_kernel(states, SIR_params):
^

  warnings.warn(NumbaPendingDeprecationWarning(msg, loc=loc))
c:\python37\lib\site-packages\numba\ir_utils.py:1969: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected list' found for argument 'states' of function 'recover_kernel'.

For more information visit http://numba.pydata.org/numba-doc/latest/reference/deprecation.html#deprecation-of-reflection-for-list-and-set-types

File "<ipython-input-120-39d03efb684e>", line 36:
@jit
def recover_kernel(states

In [123]:
pprint(num_deceased)
pprint(all_population)


{0: {0: [0, 0, 0, 0, 0],
     1: [0, 0, 0, 0, 0],
     2: [0, 0, 0, 0, 0],
     3: [0, 0, 0, 0, 0]},
 1: {0: [0, 0, 0, 0, 0],
     1: [0, 0, 0, 0, 0],
     2: [0, 0, 0, 0, 0],
     3: [0, 0, 0, 0, 0]},
 2: {0: [0, 0, 0, 0, 0],
     1: [0, 0, 0, 0, 0],
     2: [0, 0, 0, 0, 0],
     3: [0, 0, 0, 0, 0]}}
{0: [240, 130, 210, 240, 160],
 1: [260, 179, 214, 278, 186],
 2: [263, 215, 224, 293, 211]}


### Real data

In [124]:
data_dir = "../Data/Clean/"
deaths_fname = "deaths_NCHS_processed.csv"
population_fname = "population.csv"

states_abb_dict, states_abb_rev_dict, stats_abb_ord_list = read_US_states(data_dir + "states_abb.csv")
deaths_dict = read_deaths_data(data_dir + deaths_fname)
population_dict = read_population_dict(data_dir + population_fname)

adj_list = {}
A = {}
A[2009] = {}
A[2009][4] = read_travel_network(data_dir + "2009_Q4.csv", states_abb_dict, states_abb_rev_dict)[1]
for year in range(2010, 2019):
    A[year] = {}
    for quarter in range(1, 5):
        network_fname = str(year) + "_Q" + str(quarter) + ".csv"
        A[year][quarter] = read_travel_network(data_dir + network_fname, states_abb_dict, states_abb_rev_dict)[1]
A[2019] = {}
A[2019][1] = read_travel_network(data_dir + "2019_Q1.csv", states_abb_dict, states_abb_rev_dict)[1]

all_population = {}
years = [2009,2010,2011,2012,2013,2014,2015,2016,2017,2018]
for year in years:
    all_population[year] = []


for state,data in population_dict.items():
    for year, pop in data.items():
        all_population[year].append(pop)
        
A_dict = A

In [125]:
sim_time = time()

states = []
year_0 = 2009
for state_id in range(50):
    state = State(all_population[year_0][state_id])
    state.num_infected += np.random.binomial(state.num_total, p_inf)
    states.append(state)
    
# Propagate infection
num_deceased = {}

# For each year:
for year in years:
    iter_time = time()
    num_deceased[year] = {}
    # For each quarter:
    for quarter in range(1, 5):
        if year == 2009 and quarter != 4:
            continue
        if year == 2019 and quarter != 1:
            continue
        # 1. travel and infect
        # 2. Recover
        # 3. Remove deceased and store it
        # 4. Randomly infect
        print("travel")
        states = travel_and_infect_kernel(A_dict[year][quarter], states, p_transfer)
        print("recover")
        t2 = time()
        states = recover_kernel(states, SIR)
        print(time() - t2)
        print("remove dead")
        states, dead_peeps = remove_deceased_kernel(states)
        num_deceased[year][quarter] = dead_peeps
        print("random_inf")
        states = random_infection_kernel(states, p_inf)
    
    # Inject population
    try:
        pop_vec = inject_population_kernel(states, all_population[year+1])
    except KeyError:
        # We are at the end of our data
        pass
    
    iter_time = time() - iter_time
    print("Elapsed time for {}: {:.3f} s".format(year, iter_time))

sim_time = time() - sim_time
print("Total elapsed time for simulation: {:.3f} s".format(sim_time))

travel
recover
342.780779838562
remove dead
random_inf
Elapsed time for 2009: 342.991 s
travel
recover
8.780529499053955
remove dead
random_inf
travel
recover
105.74505305290222
remove dead
random_inf
travel
recover
105.70717406272888
remove dead
random_inf
travel
recover
109.34973859786987
remove dead
random_inf
Elapsed time for 2010: 329.808 s
travel
recover
8.85831904411316

KeyboardInterrupt: 